# BCGW Service URL Updater - July 27th 2021
DataBC has been working to improve web service performance and stability for public data access web services available to ArcGIS Online via BCs Map Hub. Historically we have published a single large service with over 700 layers. The single service will be replaced by multiple smaller services published by schema. Duplicate Map Image Layer items will also be deprecated in favour of the corresponding authoritative Feature Layer items. We are now ready to implement the changes.

#### Michael Dykes (Michael.Dykes@gov.bc.ca)

### 1. Import Required Libraries/Modules and ArcGIS Online Connection

In [ ]:
import os, getpass, json, time, base64
from arcgis.gis import GIS
from functools import reduce
from operator import getitem
import ipywidgets as widgets
import pandas as pd

gis = GIS('home')

### 2. Set File Folder Structure for JSON Text File Backups

In [ ]:
WorkFolderPath = '/arcgis/home/BCGW_Updates'
BackupPath = WorkFolderPath + "/AGO_JSON_Backups"
# If folder doesn't exist, create it (in AGO Notebooks Files)
if not os.path.exists(WorkFolderPath):
    os.mkdir(WorkFolderPath)
    os.mkdir(BackupPath)

### 3. Load Functions

In [ ]:
# Set JSON values in nested dictionaries
def Set_Nested_JSON_Value(dataDict, mapList, val):
    reduce(getitem, mapList[:-1], dataDict)[mapList[-1]] = val
    return dataDict

# Search nested dictionaries for JSON values, store the "path" to get there and the value itself
def Search_JSON(storage, haystack, needle, path=None):
    storage = storage
    if path is None:
        path = []
    if isinstance(haystack, dict):
        if needle in haystack:
            path.append(needle)
            toappend = path,haystack[needle]
            storage.append(toappend)
        for k, v in haystack.items():
            Search_JSON(storage, v, needle, path + [k])
    elif isinstance(haystack, list):
        for idx, v in enumerate(haystack):
            Search_JSON(storage, v, needle, path + [idx])
            
# Search the JSON for itemId, url, and baseURL keys and put the JSON 'path' and value in ReferenceList            
def Check_JSON_ItemReferences(ItemID):
    ReferenceList = []
    item = gis.content.get(ItemID)
    item_data = item.get_data()
    Search_JSON(ReferenceList,item_data,"itemId")
    Search_JSON(ReferenceList,item_data,"url")
    Search_JSON(ReferenceList,item_data,"baseURL")
    return ReferenceList
 
# Backup JSON data into text files on your harddrive (to be extra careful) you can use AGO assistant to 
# copy and paste them back into your AGO JSON if something goes wrong    
def Create_JSON_BackupFile(ItemID):
    item = gis.content.get(ItemID)
    item_data = item.get_data()    
    with open(BackupPath + "/" + ItemID + "_Backup.json", 'w') as outfile:
        json.dump(item_data, outfile)

# Restore item JSON from backup
def restore_button_click(self):
    backupfile = BackupPath + "/" + SearchDict[itemSelection.value] + "_Backup.json"
    if os.path.isfile(backupfile):
        with open(backupfile) as json_file:
            backupjson = json.load(json_file)
            item = gis.content.get(SearchDict[itemSelection.value])
            item.update(data=backupjson)
            BackupPass_text = widgets.Label(value="JSON Data Restored from Backup Successfully")
            bottom_box.children += (BackupPass_text,)     
    else:
        BackupFail_text = widgets.Label(value="No Backup Found")
        bottom_box.children += (BackupFail_text,)  

# Update AGO Item JSON based on GUI list
def update_URL_references(self):
    item = gis.content.get(SearchDict[itemSelection.value])
    item_data = item.get_data()

    Create_JSON_BackupFile(SearchDict[itemSelection.value])
    UpdateList = Checkbutton.UpdateList
    if UpdateList:
        for i in range(1,len(bottom_box.children[1:-1])):
            if bottom_box.children[i].children[1].value == UpdateList[i-1][1]:
                if bottom_box.children[i].children[3].value:
                    Set_Nested_JSON_Value(item_data,UpdateList[i-1][0],bottom_box.children[i].children[3].value)
                else:
                    Set_Nested_JSON_Value(item_data,UpdateList[i-1][0],bottom_box.children[i].children[2].value)
        item.update(data=item_data)
        UpdatePass_text = widgets.Label(value="AGO Item Updated Successfully")
        bottom_box.children = (UpdatePass_text,)  
    else:
        UpdateFail_text = widgets.Label(value="AGO Item Updated Failed")
        bottom_box.children += (UpdateFail_text,)
        
# Function to search from selection list for item with matching ItemID
def search_by_itemid(self):
    #If 'Search By ID' is populated, ignore all other boxes, check for correctly formatted ItemID
    if Search_ID.value:
        #Try to find the content in ArcGIS Online"
        try:
            item = gis.content.get(Search_ID.value)
            newselection = item.title + " | " + item.type
            itemSelection.value = newselection
        except:
            SearchFail_text = widgets.Label(value="ID Formatted Wrong, Does Not Exist, or Was Not Found")
            bottom_box.children += (SearchFail_text,)

# Create link to download the table of layers and url references to a csv file on your computer
def create_download_link(title = "Download as CSV file"):
    filename = gis.content.get(SearchDict[itemSelection.value]).title[:50] + "_BCGW_URLUpdate.csv"
    datalist =[]
    for row in bottom_box.children[1:]:
        datalist.append([row.children[0].value,row.children[1].value,row.children[2].value,row.children[3].value])
    df = pd.DataFrame(data = datalist, columns=['LAYERNAME','EXISTING_URL','SUGGESTED_URL','URL_OVERRIDE'])
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

# Executes function which populates GUI list of layers and url references (only those with "mpcm/bcgwpub" and "mpcm/bcgw" referenced)
def check_button_click(self):
    self.UpdateList = []
    self.ReferenceList = Check_JSON_ItemReferences(SearchDict[itemSelection.value])
    if self.ReferenceList:
        text1 = widgets.Label(value="Layer Name")
        text1.layout.width = "400px"   
        text2 = widgets.Label(value="Existing URL Reference")
        text2.layout.width = "400px"   
        text3 = widgets.Label(value="Suggested New URL Reference")
        text3.layout.width = "400px"  
        text4 = widgets.Label(value="New URL Suggestion Override")
        text4.layout.width = "400px" 
        RowNameLayout = widgets.HBox([text1,text2,text3,text4])
        bottom_box.children += (RowNameLayout,)
        for row in self.ReferenceList:
            if "mpcm/bcgwpub" in row[1] or "mpcm/bcgw" in row[1]:
                self.UpdateList.append(row)
                if row[1] in OldUrl_Dict:
                    if isinstance(OldUrl_Dict[row[1]][0],float):
                        value1 = widgets.Textarea(value="")  
                    else:
                        value1 = widgets.Textarea(value=OldUrl_Dict[row[1]][0],disabled=True) 
                else:
                    value1 = widgets.Textarea(value="")  
                value1.layout.height = "50px"
                value1.layout.width = "400px" 
                value2 = widgets.Textarea(value=row[1],disabled=True)
                value2.layout.height = "50px"
                value2.layout.width = "400px"    
                if row[1] in OldUrl_Dict:
                    if isinstance(OldUrl_Dict[row[1]][1],float):
                        value3 = widgets.Textarea(value=OldUrl_Dict[row[1]][1],disabled=True)
                else:
                    value3 = widgets.Textarea(value="")
                value3.layout.height = "50px"
                value3.layout.width = "400px"    
                value4 = widgets.Textarea(value=None)
                value4.layout.height = "50px"
                value4.layout.width = "400px"    
                RowLayout = widgets.HBox([value1,value2,value3,value4])
                bottom_box.children += (RowLayout,)
        UpdateButton = widgets.Button(description='Update URLs')
        UpdateButton.on_click(update_URL_references)
        link = widgets.HTML(value=create_download_link())
        bottom_box.children += (UpdateButton,link,)
        
    if not self.UpdateList:
        NoUpdate_text = widgets.Label(value="No Updates Needed/Found")
        bottom_box.children = (NoUpdate_text,)

# Watches selection list for change in selection and updates the other GUIs in response
def on_selection_change(self):
    Search_ID.value = ""
    bottom_box.children = ()
    title_text.value = "Title: " + gis.content.get(SearchDict[itemSelection.value]).title
    type_text.value = "Type: " + gis.content.get(SearchDict[itemSelection.value]).type
    owner_text.value = "Owner: " + gis.content.get(SearchDict[itemSelection.value]).owner
    created_text.value = "Created: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(SearchDict[itemSelection.value]).created/1000)))
    modified_text.value = "Modified: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(SearchDict[itemSelection.value]).modified/1000)))
    url_text.value = "<a href=" + gis.url + "/home/item.html?id=" + SearchDict[itemSelection.value] + ">Go to Item Page</a>"
    if gis.content.get(SearchDict[itemSelection.value]).get_thumbnail():
        thumb_image.visibility = 'visible'
        thumb_image.value = gis.content.get(SearchDict[itemSelection.value]).get_thumbnail()
    else:
        thumb_image.visibility = 'hidden'

# Watch checkbox to filter selection list or not
def checkbox_filter(b):
    if b["new"]:
        itemtypes_list = ["Web Map","Web Mapping Application","StoryMap","Dashboard"]
        new_item_list = []
        for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=5000):
            if item.type in itemtypes_list:
                new_item_list.append([item.title,item.id])
        sorteditem_list = sorted(new_item_list,key = lambda x:x[0])
        SearchDict = {}
        for row in sorteditem_list:
            SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
            SearchDict[SearchDisplay] = row[1]
        itemSelection.options = sorted(SearchDict.keys())
    else:
        new_item_list = []
        for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=5000):
            new_item_list.append([item.title,item.id])
        sorteditem_list = sorted(new_item_list,key = lambda x:x[0])
        SearchDict = {}
        for row in sorteditem_list:
            SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
            SearchDict[SearchDisplay] = row[1] 
        itemSelection.options = sorted(SearchDict.keys())

### 4. Build Dictionaries for Old/Deprecated URLs and Old/Deprecated ItemIDs Lookup

In [ ]:
# Read csv file from GITHub using pandas
url = 'https://raw.githubusercontent.com/MichaelDykesBC/AGO/master/AGO_BCGW_Lookup.csv'
df = pd.read_csv(url,index_col=0,encoding='cp1252')

# Empty Dictionaries to hold data
OldUrl_Dict = {}
OldItemID_Dict = {}

# Iterate through CSV lookup table
for index, row in df.iterrows():
    OldServiceTitle = index
    OldServiceURL = row[0]
    AGOTitle = row[1]
    AGOServiceURL = row[2]
    AGOItemID = row[4]
    DeprecatedItemIDs = row[5]
    DeprecatedURLs = row[6]

    OldUrl_Dict[OldServiceURL] = [AGOTitle,AGOServiceURL,AGOItemID]
    
    # Handle deprecated URLs
    if pd.notnull(DeprecatedURLs):
        if "dynamicLayer?layer=" in DeprecatedURLs:
            if DeprecatedURLs in OldUrl_Dict:
                if OldUrl_Dict[DeprecatedURLs][0][1] != AGOServiceURL:
                    toappend = AGOTitle,AGOServiceURL,AGOItemID
                    OldUrl_Dict[DeprecatedURLs].append(toappend)
            else:
                OldUrl_Dict[DeprecatedURLs] = [AGOTitle,AGOServiceURL,AGOItemID]
        else:
            for item in DeprecatedURLs.split(","):
                if item:
                    if item in OldUrl_Dict:
                        if OldUrl_Dict[item][1] != AGOServiceURL:
                            toappend = AGOTitle,AGOServiceURL,AGOItemID
                            OldUrl_Dict[item].append(toappend)
                    else:
                        OldUrl_Dict[item] = [AGOTitle,AGOServiceURL,AGOItemID]
                        
    # Handle deprecated itemIDs                    
    if pd.notnull(DeprecatedItemIDs):
        for item in DeprecatedItemIDs.split(","):
            if item:
                if item in OldItemID_Dict:
                    if OldItemID_Dict[item][1] != AGOServiceURL:
                        toappend = AGOTitle,AGOServiceURL,AGOItemID
                        OldItemID_Dict[item].append(toappend)
                else:
                    OldItemID_Dict[item] = [AGOTitle,AGOServiceURL,AGOItemID]
                    
                    
for k,v in OldUrl_Dict.items():
    print(k,v)

### 5. Build Application/GUI

In [ ]:
# Search through AGO for content owned by whomever is running this Notebook (max 5000 items) and put them in a list object
item_list = []
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=5000):
    item_list.append([item.title,item.id])

# Sort list object of AGO items alphabetically by title
sorteditem_list = sorted(item_list,key = lambda x:x[0])

# Create dictionary for selection window title + itemID to access items based of selection
SearchDict = {}
for row in sorteditem_list:
    SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
    SearchDict[SearchDisplay] = row[1]

# GUI building below
filtercheckbox = widgets.Checkbox(description='Filter List to Only Show Web Apps, Dashboards, Story Maps, and Web Maps')
filtercheckbox.layout.width = "700px"
itemSelection = widgets.Select(options=sorted(SearchDict.keys()),disabled=False)
itemSelection.layout.height = "350px"
itemSelection.layout.width = "700px" 
Search_ID = widgets.Text()
Search_ID.layout.width = "550px" 
Searchbutton = widgets.Button(description='Search by ItemID')
Searchbutton.on_click(search_by_itemid)
thumb_image = widgets.Image(value=gis.content.get(sorteditem_list[0][1]).get_thumbnail(),format="png",width=300,height=400)
title_text = widgets.Label(value="Title: " + sorteditem_list[0][0])
type_text = widgets.Label(value="Type: " + gis.content.get(sorteditem_list[0][1]).type) 
owner_text = widgets.Label(value="Owner: " + gis.content.get(sorteditem_list[0][1]).owner) 
created_text = widgets.Label(value="Created: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(sorteditem_list[0][1]).created/1000))))
modified_text = widgets.Label(value="Modified: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(sorteditem_list[0][1]).modified/1000))))
url_text = widgets.HTML(value="<a href=" + gis.url + "/home/item.html?id=" + SearchDict[itemSelection.value] + ">Go to Item Page</a>")

itemSelection.observe(on_selection_change,names='value')  
filtercheckbox.observe(checkbox_filter, names=['value'])
Checkbutton = widgets.Button(description='Check Item')
Checkbutton.on_click(check_button_click)

Restorebutton = widgets.Button(description='Restore Item JSON')
Restorebutton.on_click(restore_button_click)

search_box = widgets.HBox([Search_ID,Searchbutton])
info_box = widgets.VBox([title_text,type_text,owner_text,created_text,modified_text,url_text])
top_box = widgets.HBox([thumb_image,info_box])
buttons_box = widgets.HBox([Checkbutton,Restorebutton])
bottom_box = widgets.VBox()
widgetlist = [filtercheckbox,itemSelection,search_box,top_box,buttons_box,bottom_box]
itemGUI = widgets.VBox(widgetlist)

### 6. BCGW Service URL Updating Application

In [ ]:
display(itemGUI)